In [1]:
import os
import glob
import pandas as pd
import numpy as np

In [2]:
def checaArquivos():
    
    checaArquivoTXT = len(glob.glob(os.path.join(os.path.abspath('./input'), "*.txt")))
    checaArquivoCSV = len(glob.glob(os.path.join(os.path.abspath('./input'), "*.CSV")))
    
    if(checaArquivoTXT == 0 and checaArquivoCSV == 0):
        print("Em falta de ambos os arquivos na pasta 'input'!!", 'red')
    elif(checaArquivoTXT == 0 and checaArquivoCSV == 1):
        print("Em falta do arquivo de coleta na pasta 'input'!!", 'red')
    elif(checaArquivoTXT == 1 and checaArquivoCSV == 0):
        print("Em falta do arquivo do Virtual Age na pasta 'input'!!", 'red')
    elif(checaArquivoTXT == 2 or checaArquivoCSV == 2):
        print("Existem arquivos além dos esperados na pasta 'input'!!", 'red')
    else:
        return "Ok"

In [3]:
def divideClasses(df):
    
    condition = ((df["Local Coletado"].notna()) & (df["Local VA"].notna())) & (df["Local Coletado"] != df["Local VA"])
    PcLD = df[condition].reset_index(drop=True)
    
    PnEnI = df[df["Local Coletado"].isna()].reset_index(drop=True)
    PnEnS = df[df["Local VA"].isna()].reset_index(drop=True)
    
    return PcLD, PnEnI, PnEnS

In [4]:
def reorganizacaoDeClasses(PcLD, PnEnS, dfCompair2):
    
    comparacaoForaDoEscopo = dfCompair2['Peça'].to_list()
    
    PnEnS = PnEnS.drop(PnEnS[PnEnS['Peça'].astype('int64').isin(comparacaoForaDoEscopo)].index.to_list())
    PcLD = pd.concat([PcLD, dfCompair2], ignore_index=True)
    
    return PcLD, PnEnS

In [5]:
def comparaForaDoEscopoDoFiltro(df1, df2):
    
    df1 = df1.rename(columns = {"Peças VA" : "Peça"})
    df2 = df2[["Peça", "Local Coletado"]].astype({'Peça' : 'int64'})
        
    dfMergeColeta = df2.merge(df1, how = "left", on = "Peça")

    mergeAll = dfMergeColeta[["Peça", "Local VA", "Local Coletado"]]
    
    return mergeAll

In [6]:
def comparaNoEscopoDoFiltro(df1, df2):
    
    df1 = df1.rename(columns = {"Peças VA" : "Peça"})
    df2 = df2.rename(columns = {"Peças Coletadas" : "Peça"})

    dfMergeColeta = df2.merge(df1, how = "left", on = "Peça")
    dfMergeVA = df1.merge(df2, how = "left", on = "Peça")

    mergeAll = pd.concat([dfMergeVA, dfMergeColeta], ignore_index=True)
    mergeAll = mergeAll.astype({'Peça': 'string'})
    
    PcLD, PnEnI, PnEnS = divideClasses(mergeAll)
    
    return mergeAll

In [7]:
def defineRua(string):
    
    if string == '1':
        return 'A'
    elif string == '2':
        return 'B'
    elif string == '3':
        return 'C'
    elif string == '4':
        return 'D'
    elif string == '5':
        return 'E'
    elif string == '6':
        return 'F'
    elif string == '7':
        return 'G'
    elif string == '8':
        return 'H'
    elif string == '9':
        return 'I'
    else:
        return "ERRO"

def defineColunaEGaveta(n):
    
    col = n[0:2]
    gav = n[2]

    return " COLUNA {} GAVETA 0{}".format(col, gav)

def transformaLocal(df, coluna):
    
    df[coluna] = df[coluna].map(lambda x: "RUA " + defineRua(str(x)[0]) + defineColunaEGaveta(str(x)[1:4]))

In [8]:
def formataFiltros(filtro):
    
    return "RUA " + filtro[1] + " COLUNA " + filtro[4:6] + " GAVETA " + filtro[8:]
    

def defineRangeLocal(df, inicio, fim):
    
    inicio_formatado = formataFiltros(inicio)
    fim_formatado = formataFiltros(fim)
    
    dfFiltrado = df[df["Local VA"] >= inicio_formatado]
    dfFiltrado = dfFiltrado[dfFiltrado["Local VA"] <= fim_formatado]
    
    dfForaDoFiltro1 = df[df["Local VA"] < inicio_formatado]
    dfForaDoFiltro2 = df[df["Local VA"] > fim_formatado]
    
    dfForaDoFiltro = pd.concat([dfForaDoFiltro1, dfForaDoFiltro2], ignore_index=True)
    
    return dfFiltrado, dfForaDoFiltro

In [9]:
def coletaParaVA(df):
    df.insert(1, "Local Coletado", ['' for i in range(0, len(df.index))], allow_duplicates = True)
    
    df["Local Coletado"] = df["Peças Coletadas"].map(lambda x: int(str(x)[-6:]))
    transformaLocal(df, "Local Coletado")
    df["Peças Coletadas"] = df["Peças Coletadas"].map(lambda x: int(str(x)[:-6]))

    return df

In [10]:
def dataFrameToExcel_PcLD(df, endereco):
    
    condition = ((df["Local Coletado"].notna()) & (df["Local VA"].notna())) & (df["Local Coletado"] != df["Local VA"])
    
    filename = "Peças com Local Diferente.xlsx"
    
    df[condition].to_excel((endereco + filename), index=False)
    
def dataFrameToExcel_PnEnI(df, endereco):
    
    filename = "Peças não Encontradas no Inventário.xlsx"
    
    df[df["Local Coletado"].isna()][["Peça", "Local VA"]].to_excel((endereco + filename), index=False)
    
def dataFrameToExcel_PnEnS(df, endereco):
    
    filename = "Peças não Encontradas no Sistema.xlsx"
    
    df[df["Local VA"].isna()][["Peça", "Local Coletado"]].to_excel((endereco + filename), index=False)


def dataFrameToExcel(PcLD, PnEnI, PnEnS, endereço):
    
    PcLD = PcLD.astype({"Peça" : 'string'})
    PnEnI = PnEnI.astype({"Peça" : 'string'})
    PnEnS = PnEnS.astype({"Peça" : 'string'})
    
    dataFrameToExcel_PcLD(PcLD, endereço)
    dataFrameToExcel_PnEnI(PnEnI, endereço)
    dataFrameToExcel_PnEnS(PnEnS, endereço)

In [11]:
checaArquivos()

'Ok'

In [12]:
dfColeta = pd.read_csv(glob.glob(os.path.join(os.path.abspath('./input'), "*.txt"))[0], delimiter = ' ').rename(columns = {'Peças': 'Peças Coletadas'})

dfColeta = dfColeta[dfColeta["Peças Coletadas"].astype('string').str.len() == 16]
dfColeta = dfColeta[dfColeta["Peças Coletadas"].astype('string').str.isnumeric()]

dfColeta

,Peças Coletadas
0,1000918544101101
1,1000918542101102
2,1000918541101103
3,1000994182101104
4,1000994244101105
5,1000994451101106
6,1000994165101107
7,1000994166101108
8,1000972458101201
9,1000972457101202


In [13]:
dfVA = pd.read_csv(glob.glob(os.path.join(os.path.abspath('./input'), "*.CSV"))[0], delimiter = ';', usecols = ["Codigo 1000", "Local"]).rename(columns = {'Codigo 1000' : 'Peças VA', 'Local' : 'Local VA'}).fillna(0)[["Peças VA", "Local VA"]]
dfVA = dfVA.astype({'Peças VA' : 'int64'})

dfVA = dfVA[~dfVA['Peças VA'].astype("string").str.startswith("2000")]

dfVA

,Peças VA,Local VA
1,1000930913,DEPOSITO ALGODAO COLUNA 18 GAVETA 01
2,1000977283,DEPOSITO DE PALETES VAZIOS DEPOSITO 01 DEPOSIT...
3,1000985333,DEPOSITO DE PALETES VAZIOS DEPOSITO 01 DEPOSIT...
4,1000985334,DEPOSITO DE PALETES VAZIOS DEPOSITO 01 DEPOSIT...
5,1000957979,DEPOSITO DE PALETES VAZIOS DEPOSITO 01 DEPOSIT...
...,...,...
10359,1000850151,RUA A COLUNA 06 GAVETA 02
10360,1000850154,RUA A COLUNA 06 GAVETA 02
10361,1000850179,RUA A COLUNA 06 GAVETA 02
10363,1000916667,RUA E COLUNA 13 GAVETA 03


In [14]:
dfVAPrimario, dfVASecundario = defineRangeLocal(dfVA, "RA C01 G01", "RA C01 G02")

In [15]:
dfColeta = coletaParaVA(dfColeta)

In [16]:
dfCompair1 = comparaNoEscopoDoFiltro(dfVAPrimario, dfColeta)

In [17]:
PcLD, PnEnI, PnEnS = divideClasses(dfCompair1)

In [18]:
PcLD

,Peça,Local VA,Local Coletado


In [19]:
PnEnI

,Peça,Local VA,Local Coletado
0,1000925859,RUA A COLUNA 01 GAVETA 01,NaN
1,1000925860,RUA A COLUNA 01 GAVETA 01,NaN
2,1000925861,RUA A COLUNA 01 GAVETA 01,NaN
3,1000925862,RUA A COLUNA 01 GAVETA 01,NaN
4,1000915698,RUA A COLUNA 01 GAVETA 02,NaN
5,1000915699,RUA A COLUNA 01 GAVETA 02,NaN
6,1000915697,RUA A COLUNA 01 GAVETA 02,NaN
7,1000915690,RUA A COLUNA 01 GAVETA 02,NaN
8,1000915700,RUA A COLUNA 01 GAVETA 02,NaN
9,1000915695,RUA A COLUNA 01 GAVETA 02,NaN


In [20]:
PnEnS

,Peça,Local VA,Local Coletado
0,1000918544,NaN,RUA A COLUNA 01 GAVETA 01
1,1000918542,NaN,RUA A COLUNA 01 GAVETA 01
2,1000918541,NaN,RUA A COLUNA 01 GAVETA 01
3,1000984277,NaN,RUA A COLUNA 01 GAVETA 02
4,1000996202,NaN,RUA A COLUNA 01 GAVETA 02


In [21]:
dfCompair2 = comparaForaDoEscopoDoFiltro(dfVASecundario, PnEnS)

In [22]:
PcLD, PnEnS = reorganizacaoDeClasses(PcLD, PnEnS, dfCompair2)

In [23]:
PcLD

,Peça,Local VA,Local Coletado
0,1000918544,RUA A COLUNA 02 GAVETA 02,RUA A COLUNA 01 GAVETA 01
1,1000918542,RUA A COLUNA 02 GAVETA 02,RUA A COLUNA 01 GAVETA 01
2,1000918541,RUA A COLUNA 02 GAVETA 02,RUA A COLUNA 01 GAVETA 01
3,1000984277,DEPOSITO DE PALETES VAZIOS DEPOSITO 01 DEPOSIT...,RUA A COLUNA 01 GAVETA 02
4,1000996202,RUA E COLUNA 40 GAVETA 04,RUA A COLUNA 01 GAVETA 02


In [24]:
PnEnI

,Peça,Local VA,Local Coletado
0,1000925859,RUA A COLUNA 01 GAVETA 01,NaN
1,1000925860,RUA A COLUNA 01 GAVETA 01,NaN
2,1000925861,RUA A COLUNA 01 GAVETA 01,NaN
3,1000925862,RUA A COLUNA 01 GAVETA 01,NaN
4,1000915698,RUA A COLUNA 01 GAVETA 02,NaN
5,1000915699,RUA A COLUNA 01 GAVETA 02,NaN
6,1000915697,RUA A COLUNA 01 GAVETA 02,NaN
7,1000915690,RUA A COLUNA 01 GAVETA 02,NaN
8,1000915700,RUA A COLUNA 01 GAVETA 02,NaN
9,1000915695,RUA A COLUNA 01 GAVETA 02,NaN


In [25]:
PnEnS

,Peça,Local VA,Local Coletado


In [26]:
dataFrameToExcel(PcLD, PnEnI, PnEnS, "./output/")